In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy.linalg as LA
import random
import pandas as pd
from numpy import inf
import torch
from torch import nn
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch.optim as optim
import json
import itertools
import math
import pyprind
from random import sample

In [ ]:
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

In [87]:
############################################### DO NOT TOUCH THIS SNIPPET ############################################### 
df1 = pd.read_csv('data/trajectories-0400-0415.csv')
df2 = pd.read_csv('data/trajectories-0500-0515.csv')
df3 = pd.read_csv('data/trajectories-0515-0530.csv')

## Adding two empty columns for velocities ##
df1['V_X']=np.nan
df1['V_Y']=np.nan

lane=7
filtered_df=df1.loc[df1['Lane_ID']==lane]
list_mergingvehicles=list(filtered_df.Vehicle_ID.unique())
df1_merging_outliers= [5,210,376,388,409,458,465,743,1005,1097,1133,1215,1408,1724,1812,1909,2387,2438,1778,2911]

#print(list_mergingvehicles)


####### Filtering data #########
    ####### AND ####### 
#### Calculating Velocities ####
vehicles_actually_merging=[]
vehicle_row_indices={}
for vehicle in list_mergingvehicles:
    if vehicle not in df1_merging_outliers:
        #print(vehicle)
        vehicles_actually_merging.append(vehicle)
        temp=df1.loc[(df1['Vehicle_ID']==vehicle) & (df1['Lane_ID']==7)]
        temp_row_indices=list(temp.loc[temp['Vehicle_ID']==vehicle].index.to_numpy())
               
        temp_extra_time=[]
        extra_time_steps=15
        for i in range(extra_time_steps):
            temp_extra_time.append(temp_row_indices[len(temp_row_indices)-1]+i)
        temp_row_indices.extend(temp_extra_time)
        
        key=str(vehicle)
        vehicle_row_indices[key]=temp_row_indices

        # Velocity #

        for row_index in temp_row_indices[1:]:
                vel_X=0
                vel_Y=0                
                dt=0.1 # Sec
                                
                vel_X=(df1.iat[row_index,4]-df1.iat[row_index-1,4])/dt
                df1.iat[row_index,18]=round(vel_X,3)
                
                vel_y=(df1.iat[row_index,5]-df1.iat[row_index-1,5])/dt
                df1.iat[row_index,19]=round(vel_y,3)

dict_taining_data={}

# sample 10% of actually merging  cars #
vehicles_for_training=sample(vehicles_actually_merging,int(0.9*len(vehicles_actually_merging)))
vehicles_for_testing=[]

for vehicle in vehicles_actually_merging:
    if vehicle not in vehicles_for_training:
        vehicles_for_testing.append(vehicle)

for vehicle in vehicles_for_training:
    key=str(vehicle)
    data_index=vehicle_row_indices[key]
    ref_trajectory=[]
    for index in data_index[1:]:
        row=df1.iloc[index]
        temp_traj_point=[df1.iat[index,4],df1.iat[index,5],df1.iat[index,18],df1.iat[index,19]] # x,y,vx,vy
        ref_trajectory.append(temp_traj_point)
    dict_taining_data[key]=ref_trajectory    
    temp=df1.loc[(df1['Vehicle_ID']==vehicle)]

############### coud be used for train and test split ############### 
'''
from sklearn.model_selection import train_test_split
import numpy

with open("datafile.txt", "rb") as f:
   data = f.read().split('\n')
   data = numpy.array(data)  #convert array to numpy type array

   x_train ,x_test = train_test_split(data,test_size=0.5)       #test_size=0.5(whole_data)
'''    
############################################### DO NOT TOUCH THIS SNIPPET ############################################### 
############################################### DO NOT TOUCH THIS SNIPPET ############################################### 


'\nfrom sklearn.model_selection import train_test_split\nimport numpy\n\nwith open("datafile.txt", "rb") as f:\n   data = f.read().split(\'\n\')\n   data = numpy.array(data)  #convert array to numpy type array\n\n   x_train ,x_test = train_test_split(data,test_size=0.5)       #test_size=0.5(whole_data)\n'

In [97]:
dict_test_data={}
for vehicle in vehicles_for_testing:
    key=str(vehicle)
    data_index=vehicle_row_indices[key]
    ref_trajectory=[]
    for index in data_index[1:]:
        row=df1.iloc[index]
        temp_traj_point=[df1.iat[index,4],df1.iat[index,5],df1.iat[index,18],df1.iat[index,19]] # x,y,vx,vy
        ref_trajectory.append(temp_traj_point)
    dict_test_data[key]=ref_trajectory    
    #temp=df1.loc[(df1['Vehicle_ID']==vehicle)]

In [ ]:
### Interaction Tupple ###


In [90]:

#########################################################################
################################ AIS_gen ################################

################################ Model 1 ################################ 

class GenerateAis1(nn.Module):

    #def __init__(self,n_input,n_state,n_psi2_in=64,n_psi2_out=128):
    def __init__(self,n_input,n_state,n_psi2_in=8,n_psi2_out=16):
        super(GenerateAis1,self).__init__()
        self.PSI_layer1=nn.Linear(n_input,n_psi2_in)     # Use RELU after
        self.PSI_layer2=nn.Linear(n_psi2_in,n_psi2_out)      # Use RELU after
        self.PSI_layer3=nn.GRUCell(n_psi2_out,n_state)       # This is the Gated layer

    def forward(self,x,h):
        #x=torch.transpose(x,0,1)
        #h=torch.transpose(h,0,1)
        x=torch.relu(self.PSI_layer1(x))
        x=torch.relu(self.PSI_layer2(x))
        h=self.PSI_layer3(x,h)
        return h 

In [91]:
#########################################################################
################################ AIS_pred ###############################

################################ Model 1 ################################ 

class PredictAis1(nn.Module):
    
    #def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=8):
    def __init__(self,n_output,n_state,n_phi2_in=16,n_phi2_out=32):
        super(PredictAis1,self).__init__()
        self.PHI_layer1=nn.Linear(n_state,n_phi2_in)  # Use RELU after
        self.PHI_layer2=nn.Linear(n_phi2_in,n_phi2_out)     # Use RELU after
        self.PHI_layer3=nn.Linear(n_phi2_out,n_output)         # mean vector of a unit-variance multivariate Gaussian distribution, samples from which are used to predict the next observation

    def forward(self,x):
        x=torch.relu(self.PHI_layer1(x))
        x=torch.relu(self.PHI_layer2(x))
        output=self.PHI_layer3(x)
        return output



In [107]:
class NN_structure(object):
    def __init__(self,n_train,n_test,n_input,n_output,n_state,learning_rate,ais_gen_model,ais_pred_model,device):
        self.n_train=n_train
        self.n_test=n_test
        self.n_input=n_input
        self.n_output=n_output
        self.n_state=n_state    # Hidden state size in RNN
        self.learning_rate=learning_rate                
        self.device=device
        self.ais_gen_model=ais_gen_model
        self.ais_pred_model=ais_pred_model

        if ais_gen_model==1:
            self.gen_model=GenerateAis1(self.n_input,self.n_state).to(self.device)      
        
        if ais_pred_model==1:
            self.pred_model=PredictAis1(self.n_output,self.n_state).to(self.device)


    def save_model_weights(self,text="_"):
        name_gen_path="trained_models/ais_gen"+str(self.ais_gen_model)+"_pred"+str(self.ais_pred_model)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        torch.save(self.gen_model.state_dict(),name_gen_path)

        name_pred_path="trained_models/ais_pred"+str(self.ais_pred_model)+"_gen"+str(self.ais_gen_model)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        torch.save(self.pred_model.state_dict(),name_pred_path)

        return "saved"
    
    def load_model_weights(self,text="_"):
        name_gen_path="trained_models/ais_gen"+str(self.ais_gen_model)+"_pred"+str(self.ais_pred_model)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        #torch.save(self.gen_model.state_dict(),name_gen_path)
        self.gen_model.load_state_dict(torch.load(name_gen_path))
        self.gen_model.eval()


        name_pred_path="trained_models/ais_pred"+str(self.ais_pred_model)+"_gen"+str(self.ais_gen_model)+"_epochs"+str(self.n_train)+"_learning_rate"+str(self.learning_rate)+"_hidden_states"+str(self.n_state)+text+".pth"
        #torch.save(self.pred_model.state_dict(),name_pred_path)
        self.pred_model.load_state_dict(torch.load(name_pred_path))
        self.pred_model.eval()

        return "loaded"



    def train(self):
        train_bar=pyprind.ProgBar(self.n_train)
        self.optimizer = torch.optim.Adam(list(self.gen_model.parameters()) + list(self.pred_model.parameters()), lr=self.learning_rate, amsgrad=True)
        loss=nn.MSELoss()
        #torch.distributions.MultivariateNormal(obs_pred_next_probs[0,:], torch.eye(obs_pred_next_probs[0,:].shape[0]).to(device)).log_prob(obs[step+1,:])

        for epochs in range(self.n_train):
            train_bar.update()
            #print("epoch")
            training_car_list=list(dict_taining_data.keys())
            training_car_list=sample(training_car_list,len(training_car_list))# Random order for training

            
            for car in training_car_list:
                trajectory_data=dict_taining_data[str(car)]

                # Ais initialization
                ais=0.5*torch.ones(self.n_state,dtype=torch.float).to(self.device)

                car_loss=0
                    
                for time in range(len(trajectory_data)-1): 
                    origin_offset=np.array(trajectory_data[0][0:2]) # only for position
                    
                    input_vector=np.array(trajectory_data[time])
                    input_vector[0:2]=input_vector[0:2]-origin_offset

                    ref_vector=np.array(trajectory_data[time+1])
                    ref_vector[0:2]=ref_vector[0:2]-origin_offset

                    train_input=torch.tensor(input_vector,dtype=torch.float).to(self.device)
                    ref_output=torch.tensor(ref_vector,dtype=torch.float).to(self.device)

                    ais=self.gen_model(train_input,ais)
                    pred_output=self.pred_model(ais)
                    
                    car_loss+=loss(pred_output,ref_output)
                    
                self.optimizer.zero_grad()
                car_loss.backward()
                self.optimizer.step()

        print("saving model weights")
        status=self.save_model_weights()

        return status


    def test(self,vehicle):

        # Testing for only one car data
        # We test predictions based only newest observation
        # Later we will test it only using initial condition

        dict_test_results={}

        for epochs in range(self.n_test):
            test_car=vehicle            
        
            trajectory_data=dict_test_data[str(test_car)]

            # Ais initialization
            ais=0.5*torch.ones(self.n_state,dtype=torch.float).to(self.device)

            car_loss=0
            previous_prediction=0

            test_results_trajectory=[]
            for time in range(len(trajectory_data)-1): 

                temp_test_info=[]
                origin_offset=np.array(trajectory_data[0][0:2]) # only for position
                
                input_vector=np.array(trajectory_data[time])
                input_vector[0:2]=input_vector[0:2]-origin_offset
                
                #if time==0:
                #    temp_test_info.extend(list(input_vector))
                #    temp_test_info.extend(list(input_vector))

                ref_vector=np.array(trajectory_data[time+1])
                ref_vector[0:2]=ref_vector[0:2]-origin_offset

                temp_test_info.extend(list(ref_vector))

                train_input=torch.tensor(input_vector,dtype=torch.float).to(self.device)
                ref_output=torch.tensor(ref_vector,dtype=torch.float).to(self.device)

                ais=self.gen_model(train_input,ais)
                pred_output=self.pred_model(ais)

                pred_vector=pred_output.detach().cpu().numpy()

                temp_test_info.extend(list(pred_vector))

                test_results_trajectory.append(temp_test_info)
            
            dict_test_results[str(test_car)]=test_results_trajectory


        return dict_test_results




    
    

In [93]:
####### Parameters #######
#n_train=len(sampled_index_all_init)
n_train=500
n_test=1
n_input=4
n_output=4
n_state =24 
learning_rate=0.0003
gen_model=1
pred_model=1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




In [94]:
####### Initialise the neural network #######

network=NN_structure(n_train,n_test,n_input,n_output,n_state,learning_rate,gen_model,pred_model,device)
print("Training has started")
network.train()


Training has started


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 05:01:01


saving model weights


'saved'

In [108]:
####### Load the neural network weights from saved models #######

loaded_network=NN_structure(n_train,n_test,n_input,n_output,n_state,learning_rate,gen_model,pred_model,device)
loaded_network.load_model_weights()



'loaded'

In [113]:
test_car=vehicles_for_testing[13]
predicted_outputs=loaded_network.test(test_car)

In [115]:
# Plotting outputs #


x_ref=[]
y_ref=[]
vx_ref=[]
vy_ref=[]
x_pred=[]
y_pred=[]
vx_pred=[]
vy_pred=[]

for elem in predicted_outputs[str(test_car)]:
    x_ref.append(-elem[0])
    y_ref.append(elem[1])
    vx_ref.append(elem[2])
    vy_ref.append(elem[3])
    x_pred.append(-elem[4])
    y_pred.append(elem[5])
    vx_pred.append(elem[6])
    vy_pred.append(elem[7])

fig, ax = plt.subplots()
#plt.plot(y_ref,x_ref,'r',linewidth=2.0,label="Actual")
#plt.plot(y_pred,x_pred,'b',linewidth=2.0,label="Predicted")
plt.plot(y_ref,'r',linewidth=2.0,label="Actual")
plt.plot(y_pred,'b',linewidth=2.0,label="Predicted")
plt.savefig('graphs/Comparision_xref_xpred_justonecar_g1_p1_lr0003_g_8_16_p_16_32.png')
plt.close()